<a href="https://colab.research.google.com/github/kyaoNK/bert_asag/blob/main/BERT_ASAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
%%capture
!sudo apt -y update
# !sudo apt -y -q install libeigen3-dev libprotobuf-dev protobuf-c-compiler

In [ ]:
%%capture
!pip install -qU pytorch-lightning lightning-bolts 
!pip install -q   tqdm
!pip install -qU torch
!pip install -qU torchtext
!pip install -qU torchvision
!pip install -qU torchaudio
!pip install -q   datasets
!pip install -q   accelerate
!pip install -q   fugashi
!pip install -q   ipadic
!pip install -q   unidic-lite
!pip install -q   tqdm
!pip install -qU transformers
!pip install -q   sentencepiece
!pip install -U   sentence-transformers
!pip install natsort

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import os
sys.path.append('/content/drive/MyDrive/Colab Notebooks/my-modules')
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Datasets

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [ ]:
import os
import numpy as np
import pandas as pd
import glob
from natsort import natsorted
import torch
from sklearn.metrics import accuracy_score , f1_score , classification_report , ConfusionMatrixDisplay , confusion_matrix
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset , concatenate_datasets

In [ ]:
DATASET_DIR = '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/'
JDATASET_DIR = DATASET_DIR + 'OUTPUT/'
TDATASET_DIR = DATASET_DIR + 'OUTPUT/tutoring/'

j_datasets = load_dataset(
    path='csv',
    data_files={
        'jglue' : JDATASET_DIR + 'jglue.csv',
        'jsick' : JDATASET_DIR + 'jsick.csv'
    },
)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating jglue split: 0 examples [00:00, ? examples/s]

Generating jsick split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-befc902b745bf880/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
datafiles_dict = dict()
filepaths = natsorted(glob.glob(TDATASET_DIR + '*.csv' ))
for fp in filepaths:
    filename = os.path.basename(fp)
    key = filename[:-4]
    datafiles_dict[key] = fp
print(datafiles_dict)

t_datasets = load_dataset(
    path='csv',
    data_files=datafiles_dict
)

{'a_ca_0_train': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_0_train.csv', 'a_ca_0_valid': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_0_valid.csv', 'a_ca_1_train': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_1_train.csv', 'a_ca_1_valid': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_1_valid.csv', 'a_ca_2_train': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_2_train.csv', 'a_ca_2_valid': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_2_valid.csv', 'a_ca_3_train': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_3_train.csv', 'a_ca_3_valid': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_3_valid.csv', 'a_ca_4_train': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a

Computing checksums:  32%|###2      | 13/40 [00:05<00:10,  2.60it/s]

Extracting data files:   0%|          | 0/40 [00:00<?, ?it/s]

Generating a_ca_0_train split: 0 examples [00:00, ? examples/s]

Generating a_ca_0_valid split: 0 examples [00:00, ? examples/s]

...

Generating q_qa_3_valid split: 0 examples [00:00, ? examples/s]

Generating q_qa_4_train split: 0 examples [00:00, ? examples/s]

Generating q_qa_4_valid split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a5b1fe940638a217/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
j_datasets

DatasetDict({
    jglue: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 8907
    })
    jsick: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 9927
    })
})

In [ ]:
t_datasets

DatasetDict({
    a_ca_0_train: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 359
    })
    a_ca_0_valid: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 91
    })
    a_ca_1_train: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 354
    })
    a_ca_1_valid: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 96
    })
    a_ca_2_train: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 363
    })
    a_ca_2_valid: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 87
    })
    a_ca_3_train: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 366
    })
    a_ca_3_valid: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 84
    })
    a_ca_4_train: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num

In [ ]:
MODEL_CKPT = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT)

def tokenize(x):
    return tokenizer( x['sentenceA'] , x['sentenceB'] , padding=True , truncation=True)

tokenized_j_datasets = j_datasets.map(tokenize , batched=True , batch_size=16, remove_columns=['sentenceA', 'sentenceB'])
tokenized_t_datasets = t_datasets.map(tokenize , batched=True , batch_size=16, remove_columns=['sentenceA', 'sentenceB'])

num_labels = 7
model = (AutoModelForSequenceClassification.from_pretrained(MODEL_CKPT, num_labels=num_labels).to(device))

  0%|          | 0/557 [00:00<?, ?ba/s]

  0%|          | 0/621 [00:00<?, ?ba/s]

  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/23 [00:00<?, ?ba/s]

...

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [ ]:
tokenized_j_datasets

DatasetDict({
    jglue: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8907
    })
    jsick: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9927
    })
})

In [ ]:
tokenized_t_datasets

DatasetDict({
    a_ca_0_train: Dataset({
        features: ['label_detail_asag', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 359
    })
    a_ca_0_valid: Dataset({
        features: ['label_detail_asag', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 91
    })
    a_ca_1_train: Dataset({
        features: ['label_detail_asag', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 354
    })
    a_ca_1_valid: Dataset({
        features: ['label_detail_asag', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 96
    })
    a_ca_2_train: Dataset({
        features: ['label_detail_asag', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 363
    })
    a_ca_2_valid: Dataset({
        features: ['label_detail_asag', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 87
    })
    a_ca_3_train: Dataset({
        features: ['label_detail_asag', 'input_ids', 'token_type_ids', 'attenti

In [ ]:
def compute_metrics(pred) :
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels , preds , average='macro')
    acc = accuracy_score(labels , preds)
    return {'accuracy' : acc , 'f1' : f1}

# training args
batch_size = 16

training_args = TrainingArguments(
    output_dir="/content/",
    num_train_epochs=10,
    # learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    disable_tqdm=False,
    logging_strategy="epoch",
    push_to_hub=False,
    load_best_model_at_end=True,
    log_level="error",
    save_total_limit=10
)

# Finetuning

In [ ]:
N_EPOCHS = 3
N_KFOLD = 5

for j in [ 'no' , 'jg' , 'js' , 'jgs'] :
    if j == 'no' :
        j_dataset = None
    elif j == 'jg' :
        j_dataset = tokenized_j_datasets['jglue']
    elif j == 'js' :
        j_dataset = tokenized_j_datasets['jsick']
    elif j == 'jgs' :
        j_dataset = concatenate_datasets( [ tokenized_j_datasets['jglue'] , tokenized_j_datasets['jsick'] ] )
    
    for d in [ 'ca' , 'qa' ] :
        for t in [ 'a' , 'q' ]:
            for idx in range(N_KFOLD):
                data_name = t + '_' + d + '_' + str(idx)
                train_name = data_name + '_train'
                valid_name = data_name + '_valid'
                print(f'{j}, train: {train_name} , valid: {valid_name}')

                train_dataset = tokenized_t_datasets[train_name]
                if j_dataset != None :
                    train_dataset = concatenate_datasets( [ train_dataset , j_dataset ] )
                valid_dataset = tokenized_t_datasets[valid_name]
                print(len(train_dataset))
                print(len(valid_dataset))
                trainer = Trainer(
                    model=model,
                    args=training_args,
                    compute_metrics=compute_metrics,
                    train_dataset=train_dataset,
                    eval_dataset=valid_dataset,
                    tokenizer=tokenizer
                )
                # trainer.train()

            #     preds_output = trainer.predict(tokenized_t_datasets[valid_name])
            #     y_pred = np.argmax(preds_output.predictions, axis=1)
            #     y_true = np.array(tokenized_t_datasets[valid_name]['label']) # label
            #     if idx == 0 :
            #         preds = y_pred
            #         trues = y_true
            #     else :
            #         preds = np.append(preds, y_pred)
            #         trues = np.append(trues, y_true)

            # cr = classification_report(trues , preds)
            # print(f'\n================= {j} , {d} , {t} ===================\n')
            # print(cr)
            # print(f'===============================================\n')


no, train: a_ca_0_train , valid: a_ca_0_valid
359
91
no, train: a_ca_1_train , valid: a_ca_1_valid
354
96
no, train: a_ca_2_train , valid: a_ca_2_valid
363
87
no, train: a_ca_3_train , valid: a_ca_3_valid
366
84
no, train: a_ca_4_train , valid: a_ca_4_valid
358
92
no, train: q_ca_0_train , valid: q_ca_0_valid
360
90
no, train: q_ca_1_train , valid: q_ca_1_valid
360
90
no, train: q_ca_2_train , valid: q_ca_2_valid
360
90
no, train: q_ca_3_train , valid: q_ca_3_valid
360
90
no, train: q_ca_4_train , valid: q_ca_4_valid
360
90
no, train: a_qa_0_train , valid: a_qa_0_valid
359
91
no, train: a_qa_1_train , valid: a_qa_1_valid
354
96
no, train: a_qa_2_train , valid: a_qa_2_valid
363
87
no, train: a_qa_3_train , valid: a_qa_3_valid
366
84
no, train: a_qa_4_train , valid: a_qa_4_valid
358
92
no, train: q_qa_0_train , valid: q_qa_0_valid
360
90
no, train: q_qa_1_train , valid: q_qa_1_valid
360
90
no, train: q_qa_2_train , valid: q_qa_2_valid
360
90
no, train: q_qa_3_train , valid: q_qa_3_valid
